## Setup

In [6]:
from dotenv import load_dotenv

load_dotenv()
llm_config={"model": "gpt-4o"} 

In [7]:
import autogen

## Tasks

In [8]:
financial_tasks = [
    """What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change?""",
    """Investigate possible reasons of the stock performance.""",
    """Plot a graph comparing the stock prices over the past month.""",
]

writing_tasks = ["""Develop an engaging blog post using any information provided."""]

In [9]:
financial_assistant = autogen.AssistantAgent(
    name="Financial_assistant",
    llm_config=llm_config,
)
research_assistant = autogen.AssistantAgent(
    name="Researcher",
    llm_config=llm_config,
)
writer = autogen.AssistantAgent(
    name="writer",
    llm_config=llm_config,
    system_message="""
        You are a professional writer, known for
        your insightful and engaging articles.
        You transform complex concepts into compelling narratives.
        Reply "TERMINATE" in the end when everything is done.
        """,
)

In [10]:
user = autogen.UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "tasks",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

chat_results = await user.a_initiate_chats(  # noqa: F704
    [
        {
            "chat_id": 1,
            "recipient": financial_assistant,
            "message": financial_tasks[0],
            "silent": False,
            "summary_method": "reflection_with_llm",
        },
        {
            "chat_id": 2,
            "prerequisites": [1],
            "recipient": research_assistant,
            "message": financial_tasks[1],
            "silent": False,
            "summary_method": "reflection_with_llm",
        },
        {
            "chat_id": 3,
            "prerequisites": [1],
            "recipient": financial_assistant,
            "message": financial_tasks[2],
            "silent": False,
            "summary_method": "reflection_with_llm",
        },
        {"chat_id": 4, "prerequisites": [1, 2, 3], "recipient": writer, "silent": False, "message": writing_tasks[0]},
    ]
)


********************************************************************************
Starting a new chat....

********************************************************************************
User (to Financial_assistant):

What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change?

--------------------------------------------------------------------------------


/Users/danielboza/Documents/Projects/langgraph-experiments/.venv/lib/python3.11/site-packages/autogen/agentchat/chat.py:47: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


Financial_assistant (to User):

To solve this, we'll write a Python script that fetches the current stock prices of NVDA (NVIDIA) and TSLA (Tesla) and calculates the percentage change in their prices over the past month. We'll use the `yfinance` library to fetch the data. 

Here's the plan:
1. Fetch current stock prices for NVDA and TSLA.
2. Fetch stock prices for NVDA and TSLA from one month ago.
3. Calculate the percentage change for each stock over the past month.
4. Print the results.

Create the script and run it to get the results.

```python
# filename: stock_prices.py
import yfinance as yf
from datetime import datetime, timedelta

def get_stock_data(ticker):
    # Getting data for the past 32 days to ensure we have at least 1 month of data
    end_date = datetime.now()
    start_date = end_date - timedelta(days=32)
    
    # Downloading historical data
    stock = yf.download(ticker, start=start_date, end=end_date)
    return stock

def calculate_percentage_change(old_price, n

CancelledError: 

### NVIDIA:
- **Strong Financial Health**: High profitability, excellent revenue growth, and efficiency metrics.
- **Market Position**: Dominating in sectors like AI and semiconductors.
- **Challenges**: Regulatory scrutiny and competition. 

### Tesla:
- **Steady Performance**: Consistent profitability, lower margins compared to NVIDIA.
- **Innovation and Expansion**: Continues to innovate in the EV and energy storage markets.
- **Challenges**: Product recalls, regulatory issues, and slower growth.

Based on the financial data, both NVIDIA and Tesla remain strong players within their respective industries but face distinct challenges and growth trajectories.

In [11]:
for i, chat_res in chat_results.items():
    print(f"*****{i}th chat*******:")
    print(chat_res.summary)
    print("Human input in the middle:", chat_res.human_input)
    print("Conversation cost: ", chat_res.cost)
    print("\n\n")

NameError: name 'chat_results' is not defined